In [2]:
import pandas as pd
import numpy as np
# 用於二元目標變數
from scipy.stats import pointbiserialr

from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [15]:
# Import GDP Data
from data import OverallGeneralEcon, df

In [16]:
NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
NominalGDP = NominalGDP.rename(columns={'time':'quarter'})
#print(NominalGDP)
#print(type(NominalGDP))

In [17]:
# 將df的日期轉換為季度
df_copy = df.copy()
df_copy['quarter'] = (pd.to_datetime(df_copy['ddate'], format='%Y%m').dt.year.astype(str) + 'Q' + pd.to_datetime(df_copy['ddate'], format='%Y%m').dt.quarter.astype(str))

# 使用merge方法合併兩個data frame
df_copy = pd.merge(df_copy, NominalGDP, on='quarter', how='left').dropna()

In [19]:
df_copy.columns

Index(['ddate', 'stockid', 'direction', 'rev', 'mom', 'yoy', 'revd1', 'yoyd1',
       'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
       'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
       'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
       'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
       'revd12', 'yoyd12', 'mond12', 'quarter', 'Nominal_GDP'],
      dtype='object')

In [21]:
# variable and target columns
target_col_GDP =  ['direction']
variable_col_GDP = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12','Nominal_GDP' ]

y_GDP = df_copy[target_col_GDP] # overall y
X_GDP = df_copy[variable_col_GDP] # overall X

In [27]:
# 計算變數相關性，用以決定變數
correlation, _ = pointbiserialr(df_copy['direction'], df_copy['Nominal_GDP'])

print(correlation)

correlation2, _ = pointbiserialr(df_copy['direction'], df_copy['revd1'])
print(correlation2)

0.009671734212467519
-0.0023818022480477617


In [44]:
# 計算每個變數與目標變數的點二序列相關係數，並儲存到一個字典中
correlations = {}
for variable in variable_col_GDP:
    if variable != 'direction':
        correlation, _ = pointbiserialr(df_copy['direction'], df_copy[variable])
        correlation = round(correlation, 4)
        correlations[variable] = correlation
        #print(f'Correlation between target and {variable}: {correlation}')

#print(correlations)

# 找出相關性最大和最小的變數
max_corr_var = max(correlations, key=correlations.get)
min_corr_var = min(correlations, key=correlations.get)
print(max_corr_var)
print(f'Variable with highest correlation: {max_corr_var}, correlation: {correlations[max_corr_var]}')
print(f'Variable with lowest correlation: {min_corr_var}, correlation: {correlations[min_corr_var]}')


revd12
Variable with highest correlation: revd12, correlation: 0.0105
Variable with lowest correlation: yoyd1, correlation: -0.0105
